#### NLTK Text Pre-processing:

In [1]:
import os
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from _cpwords import compound_keywords

In [3]:
def preprocess_text(text):
    # Define a list of English stopwords
    stop_words = set(stopwords.words('english'))
    # Add specific terms to be removed
    remove_terms = {'introduct', 'literature', 'review', 'figure', 'doi', 'fig', 'table', 'conclusion', 
                    'altimg', 'gif', 'png', 'discussion', 'acknowledgment', 'appendix','http', 'copyright'}
    stop_words.update(remove_terms)

    # Initialize lemmatizer and stemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    # Convert to lowercase
    text = text.lower()
    for original, compound in compound_keywords.items():
        text = text.replace(original, compound)
    
    # Tokenize the text into sentences
    sentence_tokens = sent_tokenize(text)
    sentences = []

    # Tokenize each sentence into words and remove stopwords
    for sentence in sentence_tokens:
        words = word_tokenize(sentence)
        filtered_words = []
        for word in words:
            word = lemmatizer.lemmatize(word)
            #word = stemmer.stem(word)
            
            # Remove non-alphabetic characters except underscores and hyphens
            word = re.sub(r'[^\w\s\-]', '', word)
            
            #if word.lower() not in stop_words:
            if word.lower() not in stop_words and not word.isdigit() and 1 < len(word) <= 20:
                filtered_words.append(word)
            
        sentences.append(filtered_words)
    
    return sentences

def process_json_files(directory):
    all_sentences = []

    # Iterate through each JSON file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # Extract the original text from the JSON structure
                text = data['full-text-retrieval-response']['originalText']
                # Process the text to tokenize and remove stopwords
                processed_sentences = preprocess_text(text)
                # Append the processed sentences to the overall list
                all_sentences.extend(processed_sentences)
    
    return all_sentences

#### Train and Save Word2Vec Model:

In [4]:
directory = 'papers_json'

model = Word2Vec(
    sentences=process_json_files(directory),  
    vector_size=300,           # Size of the embedding vectors
    window=20,                 # Context window size
    min_count=2,               # Minimum occurrence in vocabulary
    workers=4                  # Number of threads for model training
)

In [7]:
model_save_path = "word2vec.model"
model.save(model_save_path)

#### Model Examination:

In [9]:
import os
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from _cpwords import compound_keywords

model = Word2Vec.load("word2vec.model")
print("Model loaded successfully.")

Model loaded successfully.


In [10]:
# Find similar words 
similar_words_rl = model.wv.most_similar('rl', topn=100)
similar_words_carbon = model.wv.most_similar('carbon', topn=20)

print("Words similar to 'rl':")
for word, similarity in similar_words_rl:
    print(f"{word}: {similarity}")

print("\nWords similar to 'carbon':")
for word, similarity in similar_words_carbon:
    print(f"{word}: {similarity}")

Words similar to 'rl':
drl: 0.8673951029777527
pre-training: 0.7499464750289917
dqn: 0.747580885887146
ddpg: 0.7355743646621704
maddpg: 0.7274594306945801
model-free: 0.7238556146621704
non-stationarity: 0.7223429679870605
ddqn: 0.7099489569664001
rl-based: 0.7083368301391602
single-agent: 0.7039849758148193
learns: 0.6998539566993713
sac: 0.6940401196479797
ddq: 0.6898158192634583
ppo: 0.6891800761222839
irl: 0.6855924129486084
dyna-pinn: 0.6837936639785767
bdq: 0.6813809871673584
gail: 0.6684640049934387
rc-ddq: 0.6657112240791321
learn: 0.6630381941795349
fashion: 0.6607140898704529
learned: 0.6595422625541687
well-trained: 0.657589852809906
policy-gradient: 0.6567992568016052
policy-based: 0.6558753252029419
policy: 0.6531656980514526
chose: 0.6458209156990051
offline: 0.6437338590621948
agent: 0.6435546278953552
mcts: 0.6267192363739014
marl: 0.6246955394744873
value-based: 0.6217694282531738
tl: 0.6204082369804382
pre-trained: 0.6198127865791321
observational: 0.6179376840591431


In [12]:
# Find similarity
similarity_1 = model.wv.similarity('control', 'operation')
print(similarity_1)

0.35463747
